In [7]:
# トークナイザーの学習は.ipynbだけで良い！？
import json
def gen():
    import json
    data = json.load(open("all.json", "r"))     # 開くpairdataセット, all.jsonだと正しくトークン化されるが、all_2だとバグる

    for d in data:
        yield d["text"]
        yield d["phoneme_text"]

In [8]:
# sample_r_1_jp.ipynbの改良版
# sample_tokenizer.ipynb
from tokenizers import Tokenizer
from tokenizers.models import BPE, Unigram, WordPiece
from tokenizers.pre_tokenizers import WhitespaceSplit
import tokenizers.decoders as decoders
tokenizer = Tokenizer(BPE(unk_token="<unk>"))
tokenizer.decoer = decoders.BPEDecoder()

tokenizer.pre_tokenizer = WhitespaceSplit()

vocab_size = 5000
special_tokens = ["<s>", "<pad>", "</s>", "<unk>"]
from tokenizers.trainers import BpeTrainer
trainer = BpeTrainer(vocab_size=vocab_size, special_tokens=special_tokens, show_progress=True)

In [9]:
# トークナイザの学習;all.jsonで行う！
tokenizer.train_from_iterator(gen(), trainer=trainer)

In [10]:
from tokenizers.processors import TemplateProcessing

tokenizer.post_processor = TemplateProcessing(
    single="<s> $A </s>",
    pair="<s> $A </s> </s> <s> $B </s>",
    special_tokens=[
        ("<s>", tokenizer.token_to_id("<s>")),
        ("</s>", tokenizer.token_to_id("</s>")),
    ],)

# tokenizer.save()は、transformersライブラリに定義されている
# トークナイザーの状態をtokenizer.jsonに保存
tokenizer.save("tokenizer.json")

In [13]:
text = "それではこれから発表を始めたいと思います"
# textのトークン化(数値化)
encoded = tokenizer.encode(text)
# 数値→文字列
tokenizer.decode(encoded.ids)

'それ では これから 発表 を 始め たい と思います'

In [12]:
# train_bart.ipynb
